# Lab 3: Modelos de clasificación en presencia de clases desbalanceadas


## ¡Bienvenido/a!

Te invitamos a realizar el primer trabajo.
- Objetivo: Para un caso práctico desarrollado en Python, aplicar diferentes técnicas de clasificación a un conjunto de datos con clases desbalanceadas, donde también se aplicarán diferentes alternativas para balancear las clases.
- Tipo de actividad: Individual
- Tipo de evaluación: Sumativa 
- Ponderación: 12%
- Puntaje: 100 puntos
- Calificación: Escala de 1 a 7, con una exigencia de 50%. La nota mínima para aprobar es 4.0.


## Introducción:

El objetivo de este laboratorio es explorar el uso de modelos de clasificación en presencia de clases desbalanceadas.
Para esto, vamos a utilizar la base de datos `post_pabellon.csv`, la cual contiene datos de pacientes que fueron
sometidos a una biopsia (extracción de tejido). La variable de interés es `HOSPITALIZACION`, la cual indica si el
paciente fue hospitalizado luego de la biopsia. 

La siguiente tabla contiene la descripción de las variables (buscar información adicional en caso de considerarlo necesario) de la base de datos:

|Variable|Descripción|Tipo|
|--------|-----------|--|
|EDAD|Edad del paciente| Numérica|
|DIABETES|Indica si el paciente tiene o no diabetes| Binaria 1/0|
|HOSPITALIZACIÓN ULTIMO MES|Indica si el paciente fue hospitalizado el mes previo al procedimiento| Binaria 1/0|
|CUP|Uso de cateter urinario al momento de la biopsia| Binaria 1/0|
|ENF. CRONICA PULMONAR OBSTRUCTIVA|Indica si el paciente tiene una efermedad crónica polmunar obstructuva| Binaria 1/0|
|VOLUMEN PROSTATICO|Incica si el volumen prostatico es mayor a 40 $cm^3$| Binaria 1/0|
|PSA| Concentración del antígeno prostático específico en la sangre| Numérica|
|BIOPSIAS PREVIAS| Indica si el paciente ha tenido biopsias previas| Binaria 1/0|
|ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS| Indica el tipo de antibiótico utilizado en la profilaxis| Categórica nominal |
|NUMERO DE MUESTRAS TOMADAS| Número de muestras tomadas en la biopsia| Numérica|
|BIOPSIA| Resultado de la biopsia| Binaria 1/0 (1: maligno, 0: benigno)|
|HOSPITALIZACION| Indica si el paciente fue hospitalizado luego de la biopsia| Binaria 1/0|


Los modelos que vamos a utilizar son los siguientes:
- Support Vector Classifier (SVC)
- Random Forest Classifier (RFC)
- Naive Bayes Classifier (NBC)

Además, vamos a utilizar las siguientes métricas para evaluar el ajuste y también medir el desempeño de los modelos (capacidad predictiva):
- `Accuracy`: Porcentaje de casos correctamente clasificados sobre el total de casos.
- `Precision`: Porcentaje de casos positivos correctamente clasificados sobre el total de casos clasificados como positivos.
- `Recall`: Porcentaje de casos positivos correctamente clasificados sobre el total de casos positivos.
- `F1-Score`: Media armónica entre Precision y Recall.
- `AUC`: Área bajo la curva ROC.

Por último, para equilibrar las clases vamos a utilizar las siguientes técnicas:
- `RandomOverSampler`: Técnica de sobremuestreo que genera muestras sintéticas de la clase minoritaria.
- `RandomUnderSampler`: Técnica de submuestreo que elimina muestras de la clase mayoritaria.
- `NearMiss`: Técnica de submuestreo que elimina muestras de la clase mayoritaria basándonos en la distancia a las muestras de la clase minoritaria.
- `ADASYN`: Técnica de sobremuestreo que genera muestras sintéticas de la clase minoritaria basándonos en la densidad de las muestras de la clase minoritaria.
- Combinación de `RandomOverSampler` y `RandomUnderSampler`.

In [37]:
# Módulos básicos para análisis y manipulación de datos
import numpy as np
import pandas as pd

# Modelos de clasificación
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB

# Módulos para evaluación de modelos
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

# Módulos para el balanceo de datos
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, ADASYN

1. Cargar la base de datos guardada en el archivo `post_pabellon.xlsx`, y guardar la data en el objeto `datos`. Luego, separar las variables de entrada en `X` y la variable respuesta en `y`. En el caso de las variables de entrada crear variables dummies para las variables categoricas nominales de más de dos niveles, para ello utilice la función `get_dummies` de la librería `pandas` con la opción `drop_first=True`.

In [15]:
datos = pd.read_excel('post_pabellon.xlsx')

In [16]:
datos

,EDAD,DIABETES,HOSPITALIZACIÓN ULTIMO MES,PSA,BIOPSIAS PREVIAS,VOLUMEN PROSTATICO,ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS,NUMERO DE MUESTRAS TOMADAS,CUP,ENF. CRONICA PULMONAR OBSTRUCTIVA,BIOPSIA,HOSPITALIZACION
0,53,0,0,4.0,0,1,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,0,1
1,56,0,0,7.7,0,1,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,0,1
2,53,0,0,7.0,0,1,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,0,1
3,65,0,0,4.3,0,0,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,0,1
4,62,0,0,7.0,0,1,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
563,57,0,0,4.8,0,0,OTROS,12,0,0,0,0
564,75,0,0,75.0,0,1,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,1,0
565,78,0,0,9.3,0,1,CEFALOSPORINA_AMINOGLUCOCIDO,12,0,0,0,0
566,67,0,0,6.0,0,1,FLUOROQUINOLONA_AMINOGLICOSIDO,12,0,0,1,0


In [ ]:
datos = None      # Variable que debe modificar
y = None          # Variable que debe modificar
X = None          # Variable que debe modificar

# your code here
raise NotImplementedError

In [17]:
datos = pd.read_excel('post_pabellon.xlsx')
X = datos.drop(columns=['HOSPITALIZACION'])
y = datos['HOSPITALIZACION']
X = pd.get_dummies(X, columns=['ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS'], drop_first=True)

X.head()
y.value_counts()

   EDAD  DIABETES  HOSPITALIZACIÓN ULTIMO MES  PSA  BIOPSIAS PREVIAS  \
0    53         0                           0  4.0                 0   
1    56         0                           0  7.7                 0   
2    53         0                           0  7.0                 0   
3    65         0                           0  4.3                 0   
4    62         0                           0  7.0                 0   

   VOLUMEN PROSTATICO  NUMERO DE MUESTRAS TOMADAS  CUP  \
0                   1                          12    0   
1                   1                          12    0   
2                   1                          12    0   
3                   0                          12    0   
4                   1                          12    0   

   ENF. CRONICA PULMONAR OBSTRUCTIVA  BIOPSIA  \
0                                  0        0   
1                                  0        0   
2                                  0        0   
3                     

In [ ]:
# Test 1 - P1: Prueba de la carga de la base de datos y de la creación de las variables de entrada y respuesta tal como se especifica.

2. Realice una descomposición aleatoria estratificada según la variable respuesta con una proporción de 70% para entrenamiento y 30% para validación. Utilice una semilla de 123 para replicar los resultados.

Debe entregar los siguientes objetos:
- `X_train`: Matriz de predictores para entrenamiento.
- `X_test`: Matriz de predictores para validación.
- `y_train`: Vector de respuesta para entrenamiento.
- `y_test`: Vector de respuesta para validación.

**OBS**: Utilice la función `train_test_split` de la librería `sklearn`.

In [ ]:
X_train = None      # Variable que debe modificar
y_train = None      # Variable que debe modificar

X_test = None       # Variable que debe modificar
y_test = None       # Variable que debe modificar

# your code here
raise NotImplementedError

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=123)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train distribution:\n{y_train.value_counts()}")
print(f"y_test distribution:\n{y_test.value_counts()}")


X_train shape: (397, 13)
X_test shape: (171, 13)
y_train distribution:
0    380
1     17
Name: HOSPITALIZACION, dtype: int64
y_test distribution:
0    164
1      7
Name: HOSPITALIZACION, dtype: int64


In [ ]:
# Test 1 - P2: Prueba de dimensiones de los datos y aleatoriedad de la separación

3. A continuación, debe realizar una **validación cruzada estratificada** con 5 folds sobre la partición de entrenamiento para comparar diferentes modelos que 
se especificarán más adelante. 

Intrucciones:
- Utilice la función `StratifiedKFold` del módulo `model_selection` de la librería `sklearn`. Este objeto debe ser entregado como argumento a la función `cross_val_score`.
- Se debe utilizar el número 123 como semilla de aleatorización de la función `StratifiedKFold`. Además utilice el argumento `shuffle=True`.
- Se deben evaluar las siguientes métricas de ajuste en cada fold: `accuracy`, `precision`, `recall`, `f1` y `roc_auc`.
- El objetivo es comparar el desempeño de los modelos en cada fold, para esto, se debe calcular la **media** de cada métrica de ajuste por modelo.
- Debe entregar un dataframe de nombre `cv_results` con la siguiente estructura:

    | |Accuracy|Precision|Recall|F1|AUC|
    |-|--------|---------|------|--|-------|
    |SV|-|-|-|-|-|
    |RF|-|-|-|-|-|
    |NB|-|-|-|-|-|

    En python:
        
    ```python
    # Dataframe vacío
    cv_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
    ```
- Los modelos deben considerar todas las variables de la base de datos y utilizar los siguientes hiperparámetros:
    - SVC: `C=15`, `kernel='poly'`, `degree=2`
    - RFC: `n_estimators=50`, `max_depth=10`, `random_state=123`, `max_samples=0.8`, `max_features='log2'`
    - NBC: `class_prior=[0.5, 0.5]`

In [ ]:
## Modelos que debe ajustar

SVC_model = SVC(C=15, kernel='poly', degree=2)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

# your code here
raise NotImplementedError

In [24]:
svc_model = SVC(C=15, kernel='poly', degree=2, random_state=123)
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
nb_model = BernoulliNB(class_prior=[0.5, 0.5])
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

def metricas(model, X, y):
    accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy').mean()
    precision = cross_val_score(model, X, y, cv=skf, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=skf, scoring='recall').mean()
    f1 = cross_val_score(model, X, y, cv=skf, scoring='f1').mean()
    auc = cross_val_score(model, X, y, cv=skf, scoring='roc_auc').mean()
    return accuracy, precision, recall, f1, auc

cv_results.loc['SV'] = metricas(svc_model, X_train, y_train)
cv_results.loc['RF'] = metricas(rf_model, X_train, y_train)
cv_results.loc['NB'] = metricas(nb_model, X_train, y_train)
print(cv_results)


C:\Users\ccana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ccana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ccana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill

    Accuracy Precision    Recall        F1       AUC
SV  0.957215       0.0       0.0       0.0  0.735307
RF  0.959747       0.3  0.133333      0.18  0.855702
NB  0.838829  0.051496  0.183333  0.078941  0.619408


In [23]:
cv_results

,Accuracy,Precision,Recall,F1,AUC
SV,0.957215,0.0,0.0,0.0,0.735307
RF,0.959747,0.3,0.133333,0.18,0.855702
NB,0.838829,0.051496,0.183333,0.078941,0.619408


In [ ]:
# Test 1 - P3: Se comparan los valores de la matriz cv_results completa.

4. De acuerdo con los resultados anteriores, ¿cuál modelo recomedaría?. 

Para ello primero debe indicar la medida de desempeño en `medida`, donde debe escoger entre 'Accuracy', 'Precision', 'F1', 'Recall' y 'AUC'. (Por ejemplo, `medida='Accuracy'`). Una vez seleccionada la medida debe indicar el modelo de mejor desempeño, de acuerdo con la validación cruzada, en el objeto `model_select`, para indicar el mejor modelo utilizar `SV`, `RF` o `NB` (Por ejemplo, `model_select='SV'`).

In [ ]:
medida = None            # Variable que debe modificar
model_select = None      # Variable que debe modificar

# your code here
raise NotImplementedError

In [ ]:
medida = 'AUC'  
model_select = 'RF'


In [ ]:
# Test 1 - P4: Elección adecuada del modelo

5. También se debe evaluar el desempeño de los modelos, es decir, su capacidad predictiva o de generalización. Tener en cuenta, que por lo general, se debe evaluar solo la generalización del modelo escogido con la muestra de entrenamiento, cuando este último se considera aceptable. Solo con fines comparativos se evaluará la generalización de los 3 modelos propuestos.


Para esto, ajuste los modelos del punto anterior con los datos de entrenamiento y evalúelos con los datos de validación. Debe entregar un dataframe de nombre `test_results` con la siguiente estructura:

| |Accuracy|Precision|Recall|F1|AUC|
|-|--------|---------|------|--|-------|
|SV|-|-|-|-|-|
|RF|-|-|-|-|-|
|NB|-|-|-|-|-|

En python:
    
```python
# Dataframe vacío
test_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
```

Considere los mismos hiperparámetros del punto anterior. En caso de división por cero se debe deja el indicar (o métrica) en 0, para ello utilice el argumente `zero_division=0`.

In [ ]:
# your code here
raise NotImplementedError

In [25]:
test_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
svc = SVC(C=15, kernel='poly', degree=2, random_state=123)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

test_results.loc['SV', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_svc)
test_results.loc['SV', 'Precision'] = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
test_results.loc['SV', 'Recall'] = metrics.recall_score(y_test, y_pred_svc, zero_division=0)
test_results.loc['SV', 'F1'] = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
test_results.loc['SV', 'AUC'] = metrics.roc_auc_score(y_test, svc.decision_function(X_test))  

rfc = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)

test_results.loc['RF', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_rfc)
test_results.loc['RF', 'Precision'] = metrics.precision_score(y_test, y_pred_rfc, zero_division=0)
test_results.loc['RF', 'Recall'] = metrics.recall_score(y_test, y_pred_rfc, zero_division=0)
test_results.loc['RF', 'F1'] = metrics.f1_score(y_test, y_pred_rfc, zero_division=0)
test_results.loc['RF', 'AUC'] = metrics.roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1])  

nbc = BernoulliNB(class_prior=[0.5, 0.5])
nbc.fit(X_train, y_train)
y_pred_nbc = nbc.predict(X_test)

test_results.loc['NB', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_nbc)
test_results.loc['NB', 'Precision'] = metrics.precision_score(y_test, y_pred_nbc, zero_division=0)
test_results.loc['NB', 'Recall'] = metrics.recall_score(y_test, y_pred_nbc, zero_division=0)
test_results.loc['NB', 'F1'] = metrics.f1_score(y_test, y_pred_nbc, zero_division=0)
test_results.loc['NB', 'AUC'] = metrics.roc_auc_score(y_test, nbc.predict_proba(X_test)[:, 1])  

test_results


,Accuracy,Precision,Recall,F1,AUC
SV,0.959064,0.0,0.0,0.0,0.719512
RF,0.959064,0.0,0.0,0.0,0.891115
NB,0.853801,0.090909,0.285714,0.137931,0.729094


In [ ]:
# Test 1 - P5: Se comparan los valores de la matriz test_results completa.

Hasta ahora, hemos trabajado con los datos "originales", es decir, sin balancear las clases. Lo que produce este problema es la inclinación de los modelos a prever la clase mayoritaria, resultando en un rendimiento deficiente. Para solucionar esto, vamos a utilizar las técnicas de remuestreo mencionadas en la introducción del laboratorio.

Además, habrá notado que la métricas: `precision` y `recall` no han tenido un buen desempeño en los modelos anteriores. Esto se debe a que estas métricas están relacionadas con la clase minoritaria. Luego, para mejorar el desempeño de estas métricas, debemos balancear las clases.

Las siguientes preguntas tienen como propósito evaluar el rendimiento de los modelos previos utilizando datos balanceados. **Aunque es posible que cada modelo mejore con ajustes más precisos, el enfoque de este laboratorio es analizar el comportamiento de los modelos al equilibrar las clases con las diferentes técnicas disponibles**.

**MUY IMPORTANTE**: para las preguntas 6, 7 y 8 debe entregar los dataframes con el mismo formato anterior, es decir:

| |Accuracy|Precision|Recall|F1|AUC|
|-|--------|---------|------|--|-------|
|SV|-|-|-|-|-|
|RF|-|-|-|-|-|
|NB|-|-|-|-|-|

6 - Realice un sobremuestreo de la clase minoritaria utilizando la técnica `RandomOverSampler` y evalúe el desempeño de los modelos con los datos balanceados. Debe replicar lo solicitado en el punto 3 y 5, pero ahora con los datos balanceados. Debe entregar los siguientes objetos:

- `cv_results_balanced`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds.
- `test_results_balanced`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación.

Considere utilizar los siguientes hiperparámetros: `random_state=123` y `sampling_strategy='minority'` en `RandomOverSampler`.

In [ ]:
# your code here
raise NotImplementedError

In [47]:
random_over = RandomOverSampler(random_state=123, sampling_strategy='minority')
X_train_balanced, y_train_balanced = random_over.fit_resample(X_train, y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_results_balanced = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

modelos = {
    'SV': SVC(C=15, kernel='poly', degree=2, random_state=123, probability=True),
    'RF': RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2'),
    'NB': BernoulliNB(class_prior=[0.5, 0.5])
}

for nombre_modelo, modelo in modelos.items():
    y_pred = np.zeros(y_train_balanced.shape)
    y_proba = np.zeros((y_train_balanced.shape[0], 2))

    for train_index, val_index in skf.split(X_train_balanced, y_train_balanced):
        X_train_fold, X_val_fold = X_train_balanced.iloc[train_index], X_train_balanced.iloc[val_index]
        y_train_fold, y_val_fold = y_train_balanced.iloc[train_index], y_train_balanced.iloc[val_index]
        modelo.fit(X_train_fold, y_train_fold)
        y_pred[val_index] = modelo.predict(X_val_fold)
        y_proba[val_index] = modelo.predict_proba(X_val_fold)

    accuracy = metrics.accuracy_score(y_train_balanced, y_pred)
    precision = metrics.precision_score(y_train_balanced, y_pred, zero_division=0)
    recall = metrics.recall_score(y_train_balanced, y_pred, zero_division=0)
    f1 = metrics.f1_score(y_train_balanced, y_pred, zero_division=0)
    auc = metrics.roc_auc_score(y_train_balanced, y_proba[:, 1])

    cv_results_balanced.loc[nombre_modelo, 'Accuracy'] = accuracy
    cv_results_balanced.loc[nombre_modelo, 'Precision'] = precision
    cv_results_balanced.loc[nombre_modelo, 'Recall'] = recall
    cv_results_balanced.loc[nombre_modelo, 'F1'] = f1
    cv_results_balanced.loc[nombre_modelo, 'AUC'] = auc

test_results_balanced = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for nombre_modelo, modelo in modelos.items():
    modelo.fit(X_train_balanced, y_train_balanced)
    y_test_pred = modelo.predict(X_test)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
    test_precision = metrics.precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = metrics.recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = metrics.f1_score(y_test, y_test_pred, zero_division=0)
    test_auc = metrics.roc_auc_score(y_test, modelo.predict_proba(X_test)[:, 1])

    test_results_balanced.loc[nombre_modelo, 'Accuracy'] = test_accuracy
    test_results_balanced.loc[nombre_modelo, 'Precision'] = test_precision
    test_results_balanced.loc[nombre_modelo, 'Recall'] = test_recall
    test_results_balanced.loc[nombre_modelo, 'F1'] = test_f1
    test_results_balanced.loc[nombre_modelo, 'AUC'] = test_auc

print("Resultados de la validación cruzada datos balanceados (random_over):")
print(cv_results_balanced)

print("\nResultados de la evaluación con el conjunto de validación datos balanceados (random_over):")
print(test_results_balanced)


Resultados de la validación cruzada datos balanceados (random_over):
    Accuracy Precision    Recall        F1       AUC
SV  0.651316  0.589147       1.0  0.741463  0.690218
RF  0.988158  0.976864       1.0  0.988296  0.998792
NB  0.664474  0.648456  0.718421  0.681648   0.74893

Resultados de la evaluación con el conjunto de validación datos balanceados (random_over):
    Accuracy Precision    Recall        F1       AUC
SV  0.345029  0.058824       1.0  0.111111  0.848432
RF  0.953216  0.428571  0.428571  0.428571  0.857143
NB  0.555556      0.04  0.428571  0.073171  0.642857


### copilot


In [55]:
random_over = RandomOverSampler(random_state=123, sampling_strategy='minority')
X_resampled, y_resampled = random_over.fit_resample(X_train, y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_results_balanced = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

def metricas(model, X, y):
    accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy').mean()
    precision = cross_val_score(model, X, y, cv=skf, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=skf, scoring='recall').mean()
    f1 = cross_val_score(model, X, y, cv=skf, scoring='f1').mean()
    auc = cross_val_score(model, X, y, cv=skf, scoring='roc_auc').mean()
    return accuracy, precision, recall, f1, auc


svc_model = SVC(C=15, kernel='poly', degree=2, random_state=123)
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
nb_model = BernoulliNB(class_prior=[0.5, 0.5])

cv_results_balanced.loc['SV'] = metricas(svc_model, X_resampled, y_resampled)
cv_results_balanced.loc['RF'] = metricas(rf_model, X_resampled, y_resampled)
cv_results_balanced.loc['NB'] = metricas(nb_model, X_resampled, y_resampled)

print(cv_results_balanced)

test_results_balanced = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])


svc_model.fit(X_resampled, y_resampled)
y_pred_svc = svc_model.predict(X_test)

test_results_balanced.loc['SV', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_svc)
test_results_balanced.loc['SV', 'Precision'] = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced.loc['SV', 'Recall'] = metrics.recall_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced.loc['SV', 'F1'] = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced.loc['SV', 'AUC'] = metrics.roc_auc_score(y_test, svc_model.decision_function(X_test))

rf_model.fit(X_resampled, y_resampled)
y_pred_rfc = rf_model.predict(X_test)

test_results_balanced.loc['RF', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_rfc)
test_results_balanced.loc['RF', 'Precision'] = metrics.precision_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced.loc['RF', 'Recall'] = metrics.recall_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced.loc['RF', 'F1'] = metrics.f1_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced.loc['RF', 'AUC'] = metrics.roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])


nb_model.fit(X_resampled, y_resampled)
y_pred_nbc = nb_model.predict(X_test)

test_results_balanced.loc['NB', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_nbc)
test_results_balanced.loc['NB', 'Precision'] = metrics.precision_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced.loc['NB', 'Recall'] = metrics.recall_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced.loc['NB', 'F1'] = metrics.f1_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced.loc['NB', 'AUC'] = metrics.roc_auc_score(y_test, nb_model.predict_proba(X_test)[:, 1])

print(test_results_balanced)


    Accuracy Precision    Recall        F1       AUC
SV  0.651316  0.589468       1.0  0.741624  0.691118
RF  0.988158  0.976954       1.0   0.98832  0.998979
NB  0.664474  0.650265  0.718421  0.682299  0.757185
    Accuracy Precision    Recall        F1       AUC
SV  0.345029  0.058824       1.0  0.111111  0.848432
RF  0.953216  0.428571  0.428571  0.428571  0.857143
NB  0.555556      0.04  0.428571  0.073171  0.642857


In [ ]:
# Test 1 - P6: Se comparan los valores de la matriz cv_results_balanced completa.

In [ ]:
# Test 2 - P6: Se comparan los valores de la matriz test_results_balanced completa.

7 - Nuevamente realice la validación cruzada y la evaluación del conjunto de prueba para los modelos de clasificación, pero esta vez utilizando la técnica `RandomUnderSampler` para balancear las clases. Debe entregar los siguientes objetos:

- `cv_results_balanced2`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds.
- `test_results_balanced2`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación.

Considere utilizar los siguientes hiperparámetros: `random_state=123` y `sampling_strategy='majority'` en `RandomUnderSampler`.

In [ ]:
# your code here
raise NotImplementedError

In [48]:
random_under = RandomUnderSampler(random_state=123, sampling_strategy='majority')
X_train_balanced2, y_train_balanced2 = random_under.fit_resample(X_train, y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_results_balanced2 = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

modelos = {
    'SV': SVC(C=15, kernel='poly', degree=2, random_state=123, probability=True),
    'RF': RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2'),
    'NB': BernoulliNB(class_prior=[0.5, 0.5])
}

for nombre_modelo, modelo in modelos.items():
    y_pred = np.zeros(y_train_balanced2.shape)
    y_proba = np.zeros((y_train_balanced2.shape[0], 2))

    for train_index, val_index in skf.split(X_train_balanced2, y_train_balanced2):
        X_train_fold, X_val_fold = X_train_balanced2.iloc[train_index], X_train_balanced2.iloc[val_index]
        y_train_fold, y_val_fold = y_train_balanced2.iloc[train_index], y_train_balanced2.iloc[val_index]
        modelo.fit(X_train_fold, y_train_fold)
        y_pred[val_index] = modelo.predict(X_val_fold)
        y_proba[val_index] = modelo.predict_proba(X_val_fold)

    accuracy = metrics.accuracy_score(y_train_balanced2, y_pred)
    precision = metrics.precision_score(y_train_balanced2, y_pred, zero_division=0)
    recall = metrics.recall_score(y_train_balanced2, y_pred, zero_division=0)
    f1 = metrics.f1_score(y_train_balanced2, y_pred, zero_division=0)
    auc = metrics.roc_auc_score(y_train_balanced2, y_proba[:, 1])

    cv_results_balanced2.loc[nombre_modelo, 'Accuracy'] = accuracy
    cv_results_balanced2.loc[nombre_modelo, 'Precision'] = precision
    cv_results_balanced2.loc[nombre_modelo, 'Recall'] = recall
    cv_results_balanced2.loc[nombre_modelo, 'F1'] = f1
    cv_results_balanced2.loc[nombre_modelo, 'AUC'] = auc

test_results_balanced2 = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

for nombre_modelo, modelo in modelos.items():
    modelo.fit(X_train_balanced2, y_train_balanced2)
    y_test_pred = modelo.predict(X_test)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
    test_precision = metrics.precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = metrics.recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = metrics.f1_score(y_test, y_test_pred, zero_division=0)
    test_auc = metrics.roc_auc_score(y_test, modelo.predict_proba(X_test)[:, 1])

    test_results_balanced2.loc[nombre_modelo, 'Accuracy'] = test_accuracy
    test_results_balanced2.loc[nombre_modelo, 'Precision'] = test_precision
    test_results_balanced2.loc[nombre_modelo, 'Recall'] = test_recall
    test_results_balanced2.loc[nombre_modelo, 'F1'] = test_f1
    test_results_balanced2.loc[nombre_modelo, 'AUC'] = test_auc

print("Resultados de la validación cruzada datos balanceados (random_under):")
print(cv_results_balanced2)

print("\nResultados de la evaluación con el conjunto de validación datos balanceados (random_under):")
print(test_results_balanced2)


Resultados de la validación cruzada datos balanceados (random_under):
    Accuracy Precision    Recall        F1       AUC
SV  0.470588  0.470588  0.470588  0.470588  0.557093
RF  0.617647  0.590909  0.764706  0.666667  0.631488
NB  0.382353  0.409091  0.529412  0.461538  0.224913

Resultados de la evaluación con el conjunto de validación datos balanceados (random_under):
    Accuracy Precision    Recall        F1       AUC
SV  0.345029  0.051282  0.857143  0.096774  0.454704
RF  0.777778  0.121951  0.714286  0.208333  0.791812
NB  0.526316  0.059524  0.714286   0.10989   0.70122


### copilot

In [56]:
random_under = RandomUnderSampler(random_state=123, sampling_strategy='majority')
X_resampled, y_resampled = random_under.fit_resample(X_train, y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_results_balanced2 = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

def metricas(model, X, y):
    accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy').mean()
    precision = cross_val_score(model, X, y, cv=skf, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=skf, scoring='recall').mean()
    f1 = cross_val_score(model, X, y, cv=skf, scoring='f1').mean()
    auc = cross_val_score(model, X, y, cv=skf, scoring='roc_auc').mean()
    return accuracy, precision, recall, f1, auc

svc_model = SVC(C=15, kernel='poly', degree=2, random_state=123)
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
nb_model = BernoulliNB(class_prior=[0.5, 0.5])
cv_results_balanced2.loc['SV'] = metricas(svc_model, X_resampled, y_resampled)
cv_results_balanced2.loc['RF'] = metricas(rf_model, X_resampled, y_resampled)
cv_results_balanced2.loc['NB'] = metricas(nb_model, X_resampled, y_resampled)

print(cv_results_balanced2)

test_results_balanced2 = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
svc_model.fit(X_resampled, y_resampled)
y_pred_svc = svc_model.predict(X_test)

test_results_balanced2.loc['SV', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_svc)
test_results_balanced2.loc['SV', 'Precision'] = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced2.loc['SV', 'Recall'] = metrics.recall_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced2.loc['SV', 'F1'] = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced2.loc['SV', 'AUC'] = metrics.roc_auc_score(y_test, svc_model.decision_function(X_test))
rf_model.fit(X_resampled, y_resampled)
y_pred_rfc = rf_model.predict(X_test)
test_results_balanced2.loc['RF', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_rfc)
test_results_balanced2.loc['RF', 'Precision'] = metrics.precision_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced2.loc['RF', 'Recall'] = metrics.recall_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced2.loc['RF', 'F1'] = metrics.f1_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced2.loc['RF', 'AUC'] = metrics.roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])

nb_model.fit(X_resampled, y_resampled)
y_pred_nbc = nb_model.predict(X_test)
test_results_balanced2.loc['NB', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_nbc)
test_results_balanced2.loc['NB', 'Precision'] = metrics.precision_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced2.loc['NB', 'Recall'] = metrics.recall_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced2.loc['NB', 'F1'] = metrics.f1_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced2.loc['NB', 'AUC'] = metrics.roc_auc_score(y_test, nb_model.predict_proba(X_test)[:, 1])

print(test_results_balanced2)


    Accuracy Precision    Recall        F1       AUC
SV  0.466667  0.419048  0.483333  0.415238       0.4
RF  0.609524  0.497619  0.733333  0.591169  0.616667
NB   0.37619      0.42  0.516667  0.450794  0.236111
    Accuracy Precision    Recall        F1       AUC
SV  0.345029  0.051282  0.857143  0.096774  0.545296
RF  0.777778  0.121951  0.714286  0.208333  0.791812
NB  0.526316  0.059524  0.714286   0.10989   0.70122


In [ ]:
# Test 1 - P7: Se comparan los valores de la matriz cv_results_balanced2 completa.

In [ ]:
# Test 2 - P7: Se comparan los valores de la matriz test_results_balanced2 completa.

8 - Por último, nuevamente balancear las clases, pero esta vez utilizando las siguientes técnicas de balanceo de clases:
- `NearMiss`
- `ADASYN`
- Combinación de `RandomOverSampler` y `RandomUnderSampler` con `sampling_strategy=0.5` en ambos.

Los nombres de los objetos deben ser los siguientes:
- `cv_results_balanced_nm`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `NearMiss`.
- `test_results_balanced_nm`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `NearMiss`.
- `cv_results_balanced_ad`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `ADASYN`.
- `test_results_balanced_ad`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `ADASYN`.
- `cv_results_balanced_comb`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `RandomOverSampler` y `RandomUnderSampler`.
- `test_results_balanced_comb`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `RandomOverSampler` y `RandomUnderSampler`.

Debe utilizar los siguientes hiperparámetros para cada técnica de balanceo:
- `NearMiss`: `sampling_strategy='majority'`
- `ADASYN`: `random_state=123`, `sampling_strategy='minority'`
- `RandomOverSampler` y `RandomUnderSampler`: `random_state=123`, `sampling_strategy=0.5`

In [ ]:
# your code here
raise NotImplementedError

In [52]:
SV_model = SVC(C=15, kernel='poly', degree=2, random_state=123, probability=True)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

modelos = {
    'SV': SV_model,
    'RF': RF_model,
    'NB': NB_model
}

modelos = {
    'SV': SV_model,  
    'RF': RF_model,  
    'NB': NB_model   
}

skf = StratifiedKFold(n_splits=5)


nearmiss = NearMiss(sampling_strategy='majority')
X_train_balanced_nm, y_train_balanced_nm = nearmiss.fit_resample(X_train, y_train)

cv_results_balanced_nm = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
for nombre_modelo, modelo in modelos.items():
    y_pred = np.zeros(y_train_balanced_nm.shape)
    y_proba = np.zeros((y_train_balanced_nm.shape[0], 2))
    for train_index, val_index in skf.split(X_train_balanced_nm, y_train_balanced_nm):
        X_train_fold, X_val_fold = X_train_balanced_nm.iloc[train_index], X_train_balanced_nm.iloc[val_index]
        y_train_fold, y_val_fold = y_train_balanced_nm.iloc[train_index], y_train_balanced_nm.iloc[val_index]
        modelo.fit(X_train_fold, y_train_fold)
        y_pred[val_index] = modelo.predict(X_val_fold)
        y_proba[val_index] = modelo.predict_proba(X_val_fold)
    accuracy = metrics.accuracy_score(y_train_balanced_nm, y_pred)
    precision = metrics.precision_score(y_train_balanced_nm, y_pred, zero_division=0)
    recall = metrics.recall_score(y_train_balanced_nm, y_pred, zero_division=0)
    f1 = metrics.f1_score(y_train_balanced_nm, y_pred, zero_division=0)
    auc = metrics.roc_auc_score(y_train_balanced_nm, y_proba[:, 1])
    cv_results_balanced_nm.loc[nombre_modelo, 'Accuracy'] = accuracy
    cv_results_balanced_nm.loc[nombre_modelo, 'Precision'] = precision
    cv_results_balanced_nm.loc[nombre_modelo, 'Recall'] = recall
    cv_results_balanced_nm.loc[nombre_modelo, 'F1'] = f1
    cv_results_balanced_nm.loc[nombre_modelo, 'AUC'] = auc

test_results_balanced_nm = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
for nombre_modelo, modelo in modelos.items():
    modelo.fit(X_train_balanced_nm, y_train_balanced_nm)
    y_test_pred = modelo.predict(X_test)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
    test_precision = metrics.precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = metrics.recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = metrics.f1_score(y_test, y_test_pred, zero_division=0)
    test_auc = metrics.roc_auc_score(y_test, modelo.predict_proba(X_test)[:, 1])
    test_results_balanced_nm.loc[nombre_modelo, 'Accuracy'] = test_accuracy
    test_results_balanced_nm.loc[nombre_modelo, 'Precision'] = test_precision
    test_results_balanced_nm.loc[nombre_modelo, 'Recall'] = test_recall
    test_results_balanced_nm.loc[nombre_modelo, 'F1'] = test_f1
    test_results_balanced_nm.loc[nombre_modelo, 'AUC'] = test_auc

print("Resultados de la validación cruzada con NearMiss:")
print(cv_results_balanced_nm)
print("\nResultados de la evaluación con NearMiss:")
print(test_results_balanced_nm)

adasyn = ADASYN(random_state=123, sampling_strategy='minority')
X_train_balanced_ad, y_train_balanced_ad = adasyn.fit_resample(X_train, y_train)

cv_results_balanced_ad = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
for nombre_modelo, modelo in modelos.items():
    y_pred = np.zeros(y_train_balanced_ad.shape)
    y_proba = np.zeros((y_train_balanced_ad.shape[0], 2))
    for train_index, val_index in skf.split(X_train_balanced_ad, y_train_balanced_ad):
        X_train_fold, X_val_fold = X_train_balanced_ad.iloc[train_index], X_train_balanced_ad.iloc[val_index]
        y_train_fold, y_val_fold = y_train_balanced_ad.iloc[train_index], y_train_balanced_ad.iloc[val_index]
        modelo.fit(X_train_fold, y_train_fold)
        y_pred[val_index] = modelo.predict(X_val_fold)
        y_proba[val_index] = modelo.predict_proba(X_val_fold)
    accuracy = metrics.accuracy_score(y_train_balanced_ad, y_pred)
    precision = metrics.precision_score(y_train_balanced_ad, y_pred, zero_division=0)
    recall = metrics.recall_score(y_train_balanced_ad, y_pred, zero_division=0)
    f1 = metrics.f1_score(y_train_balanced_ad, y_pred, zero_division=0)
    auc = metrics.roc_auc_score(y_train_balanced_ad, y_proba[:, 1])
    cv_results_balanced_ad.loc[nombre_modelo, 'Accuracy'] = accuracy
    cv_results_balanced_ad.loc[nombre_modelo, 'Precision'] = precision
    cv_results_balanced_ad.loc[nombre_modelo, 'Recall'] = recall
    cv_results_balanced_ad.loc[nombre_modelo, 'F1'] = f1
    cv_results_balanced_ad.loc[nombre_modelo, 'AUC'] = auc

test_results_balanced_ad = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
for nombre_modelo, modelo in modelos.items():
    modelo.fit(X_train_balanced_ad, y_train_balanced_ad)
    y_test_pred = modelo.predict(X_test)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
    test_precision = metrics.precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = metrics.recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = metrics.f1_score(y_test, y_test_pred, zero_division=0)
    test_auc = metrics.roc_auc_score(y_test, modelo.predict_proba(X_test)[:, 1])
    test_results_balanced_ad.loc[nombre_modelo, 'Accuracy'] = test_accuracy
    test_results_balanced_ad.loc[nombre_modelo, 'Precision'] = test_precision
    test_results_balanced_ad.loc[nombre_modelo, 'Recall'] = test_recall
    test_results_balanced_ad.loc[nombre_modelo, 'F1'] = test_f1
    test_results_balanced_ad.loc[nombre_modelo, 'AUC'] = test_auc

print("Resultados de la validación cruzada con ADASYN:")
print(cv_results_balanced_ad)
print("\nResultados de la evaluación con ADASYN:")
print(test_results_balanced_ad)

random_over = RandomOverSampler(random_state=123, sampling_strategy=0.5)
X_train_oversampled, y_train_oversampled = random_over.fit_resample(X_train, y_train)

random_under = RandomUnderSampler(random_state=123, sampling_strategy=0.5)
X_train_balanced_comb, y_train_balanced_comb = random_under.fit_resample(X_train_oversampled, y_train)

cv_results_balanced_comb = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
for nombre_modelo, modelo in modelos.items():
    y_pred = np.zeros(y_train_balanced_comb.shape)
    y_proba = np.zeros((y_train_balanced_comb.shape[0], 2))
    for train_index, val_index in skf.split(X_train_balanced_comb, y_train_balanced_comb):
        X_train_fold, X_val_fold = X_train_balanced_comb.iloc[train_index], X_train_balanced_comb.iloc[val_index]
        y_train_fold, y_val_fold = y_train_balanced_comb.iloc[train_index], y_train_balanced_comb.iloc[val_index]
        modelo.fit(X_train_fold, y_train_fold)
        y_pred[val_index] = modelo.predict(X_val_fold)
        y_proba[val_index] = modelo.predict_proba(X_val_fold)
    accuracy = metrics.accuracy_score(y_train_balanced_comb, y_pred)
    precision = metrics.precision_score(y_train_balanced_comb, y_pred, zero_division=0)
    recall = metrics.recall_score(y_train_balanced_comb, y_pred, zero_division=0)
    f1 = metrics.f1_score(y_train_balanced_comb, y_pred, zero_division=0)
    auc = metrics.roc_auc_score(y_train_balanced_comb, y_proba[:, 1])
    cv_results_balanced_comb.loc[nombre_modelo, 'Accuracy'] = accuracy
    cv_results_balanced_comb.loc[nombre_modelo, 'Precision'] = precision
    cv_results_balanced_comb.loc[nombre_modelo, 'Recall'] = recall
    cv_results_balanced_comb.loc[nombre_modelo, 'F1'] = f1
    cv_results_balanced_comb.loc[nombre_modelo, 'AUC'] = auc

test_results_balanced_comb = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
for nombre_modelo, modelo in modelos.items():
    modelo.fit(X_train_balanced_comb, y_train_balanced_comb)
    y_test_pred = modelo.predict(X_test)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
    test_precision = metrics.precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = metrics.recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = metrics.f1_score(y_test, y_test_pred, zero_division=0)
    test_auc = metrics.roc_auc_score(y_test, modelo.predict_proba(X_test)[:, 1])
    test_results_balanced_comb.loc[nombre_modelo, 'Accuracy'] = test_accuracy
    test_results_balanced_comb.loc[nombre_modelo, 'Precision'] = test_precision
    test_results_balanced_comb.loc[nombre_modelo, 'Recall'] = test_recall
    test_results_balanced_comb.loc[nombre_modelo, 'F1'] = test_f1
    test_results_balanced_comb.loc[nombre_modelo, 'AUC'] = test_auc

print("Resultados de la validación cruzada con combinación de random_over y random_under:")
print(cv_results_balanced_comb)
print("\nResultados de la evaluación con combinación de random_over y random_under:")
print(test_results_balanced_comb)


Resultados de la validación cruzada con NearMiss:
    Accuracy Precision    Recall   F1       AUC
SV  0.588235  0.636364  0.411765  0.5  0.422145
RF  0.647059  0.692308  0.529412  0.6  0.614187
NB  0.558824     0.625  0.294118  0.4  0.612457

Resultados de la evaluación con NearMiss:
    Accuracy Precision    Recall        F1       AUC
SV  0.362573  0.044643  0.714286  0.084034  0.512195
RF  0.467836  0.053191  0.714286   0.09901  0.665505
NB  0.748538  0.071429  0.428571  0.122449  0.663763
Resultados de la validación cruzada con ADASYN:
    Accuracy Precision    Recall        F1       AUC
SV   0.57971  0.559289  0.746702  0.639548  0.676212
RF  0.919631  0.927419   0.91029  0.918775  0.966664
NB  0.760211  0.701431  0.905013  0.790323  0.792386

Resultados de la evaluación con ADASYN:
    Accuracy Precision    Recall        F1       AUC
SV  0.432749  0.067308       1.0  0.126126  0.830139
RF  0.894737  0.238095  0.714286  0.357143  0.844512
NB   0.54386  0.050633  0.571429  0.093023 

### COPILOT

In [57]:
nearmiss = NearMiss(sampling_strategy='majority')
X_resampled_nm, y_resampled_nm = nearmiss.fit_resample(X_train, y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_results_balanced_nm = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

def metricas(model, X, y):
    accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy').mean()
    precision = cross_val_score(model, X, y, cv=skf, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=skf, scoring='recall').mean()
    f1 = cross_val_score(model, X, y, cv=skf, scoring='f1').mean()
    auc = cross_val_score(model, X, y, cv=skf, scoring='roc_auc').mean()
    return accuracy, precision, recall, f1, auc

svc_model = SVC(C=15, kernel='poly', degree=2, random_state=123)
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
nb_model = BernoulliNB(class_prior=[0.5, 0.5])
cv_results_balanced_nm.loc['SV'] = metricas(svc_model, X_resampled_nm, y_resampled_nm)
cv_results_balanced_nm.loc['RF'] = metricas(rf_model, X_resampled_nm, y_resampled_nm)
cv_results_balanced_nm.loc['NB'] = metricas(nb_model, X_resampled_nm, y_resampled_nm)

print(cv_results_balanced_nm)

test_results_balanced_nm = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
svc_model.fit(X_resampled_nm, y_resampled_nm)
y_pred_svc = svc_model.predict(X_test)
test_results_balanced_nm.loc['SV', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_svc)
test_results_balanced_nm.loc['SV', 'Precision'] = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_nm.loc['SV', 'Recall'] = metrics.recall_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_nm.loc['SV', 'F1'] = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_nm.loc['SV', 'AUC'] = metrics.roc_auc_score(y_test, svc_model.decision_function(X_test))
rf_model.fit(X_resampled_nm, y_resampled_nm)
y_pred_rfc = rf_model.predict(X_test)
test_results_balanced_nm.loc['RF', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_rfc)
test_results_balanced_nm.loc['RF', 'Precision'] = metrics.precision_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_nm.loc['RF', 'Recall'] = metrics.recall_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_nm.loc['RF', 'F1'] = metrics.f1_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_nm.loc['RF', 'AUC'] = metrics.roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])
nb_model.fit(X_resampled_nm, y_resampled_nm)
y_pred_nbc = nb_model.predict(X_test)
test_results_balanced_nm.loc['NB', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_nbc)
test_results_balanced_nm.loc['NB', 'Precision'] = metrics.precision_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_nm.loc['NB', 'Recall'] = metrics.recall_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_nm.loc['NB', 'F1'] = metrics.f1_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_nm.loc['NB', 'AUC'] = metrics.roc_auc_score(y_test, nb_model.predict_proba(X_test)[:, 1])

print(test_results_balanced_nm)


adasyn = ADASYN(random_state=123, sampling_strategy='minority')
X_resampled_ad, y_resampled_ad = adasyn.fit_resample(X_train, y_train)
cv_results_balanced_ad = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
cv_results_balanced_ad.loc['SV'] = metricas(svc_model, X_resampled_ad, y_resampled_ad)
cv_results_balanced_ad.loc['RF'] = metricas(rf_model, X_resampled_ad, y_resampled_ad)
cv_results_balanced_ad.loc['NB'] = metricas(nb_model, X_resampled_ad, y_resampled_ad)

print(cv_results_balanced_ad)
test_results_balanced_ad = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
svc_model.fit(X_resampled_ad, y_resampled_ad)
y_pred_svc = svc_model.predict(X_test)
test_results_balanced_ad.loc['SV', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_svc)
test_results_balanced_ad.loc['SV', 'Precision'] = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_ad.loc['SV', 'Recall'] = metrics.recall_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_ad.loc['SV', 'F1'] = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_ad.loc['SV', 'AUC'] = metrics.roc_auc_score(y_test, svc_model.decision_function(X_test))
rf_model.fit(X_resampled_ad, y_resampled_ad)
y_pred_rfc = rf_model.predict(X_test)
test_results_balanced_ad.loc['RF', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_rfc)
test_results_balanced_ad.loc['RF', 'Precision'] = metrics.precision_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_ad.loc['RF', 'Recall'] = metrics.recall_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_ad.loc['RF', 'F1'] = metrics.f1_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_ad.loc['RF', 'AUC'] = metrics.roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])
nb_model.fit(X_resampled_ad, y_resampled_ad)
y_pred_nbc = nb_model.predict(X_test)
test_results_balanced_ad.loc['NB', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_nbc)
test_results_balanced_ad.loc['NB', 'Precision'] = metrics.precision_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_ad.loc['NB', 'Recall'] = metrics.recall_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_ad.loc['NB', 'F1'] = metrics.f1_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_ad.loc['NB', 'AUC'] = metrics.roc_auc_score(y_test, nb_model.predict_proba(X_test)[:, 1])

print(test_results_balanced_ad)


random_over = RandomOverSampler(random_state=123, sampling_strategy=0.5)
random_under = RandomUnderSampler(random_state=123, sampling_strategy=0.5)
X_resampled_ros, y_resampled_ros = random_over.fit_resample(X_train, y_train)
X_resampled_comb, y_resampled_comb = random_under.fit_resample(X_resampled_ros, y_resampled_ros)
cv_results_balanced_comb = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
cv_results_balanced_comb.loc['SV'] = metricas(svc_model, X_resampled_comb, y_resampled_comb)
cv_results_balanced_comb.loc['RF'] = metricas(rf_model, X_resampled_comb, y_resampled_comb)
cv_results_balanced_comb.loc['NB'] = metricas(nb_model, X_resampled_comb, y_resampled_comb)

print(cv_results_balanced_comb)

test_results_balanced_comb = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
svc_model.fit(X_resampled_comb, y_resampled_comb)
y_pred_svc = svc_model.predict(X_test)
test_results_balanced_comb.loc['SV', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_svc)
test_results_balanced_comb.loc['SV', 'Precision'] = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_comb.loc['SV', 'Recall'] = metrics.recall_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_comb.loc['SV', 'F1'] = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
test_results_balanced_comb.loc['SV', 'AUC'] = metrics.roc_auc_score(y_test, svc_model.decision_function(X_test))
rf_model.fit(X_resampled_comb, y_resampled_comb)
y_pred_rfc = rf_model.predict(X_test)
test_results_balanced_comb.loc['RF', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_rfc)
test_results_balanced_comb.loc['RF', 'Precision'] = metrics.precision_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_comb.loc['RF', 'Recall'] = metrics.recall_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_comb.loc['RF', 'F1'] = metrics.f1_score(y_test, y_pred_rfc, zero_division=0)
test_results_balanced_comb.loc['RF', 'AUC'] = metrics.roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])
nb_model.fit(X_resampled_comb, y_resampled_comb)
y_pred_nbc = nb_model.predict(X_test)
test_results_balanced_comb.loc['NB', 'Accuracy'] = metrics.accuracy_score(y_test, y_pred_nbc)
test_results_balanced_comb.loc['NB', 'Precision'] = metrics.precision_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_comb.loc['NB', 'Recall'] = metrics.recall_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_comb.loc['NB', 'F1'] = metrics.f1_score(y_test, y_pred_nbc, zero_division=0)
test_results_balanced_comb.loc['NB', 'AUC'] = metrics.roc_auc_score(y_test, nb_model.predict_proba(X_test)[:, 1])

print(test_results_balanced_comb)


C:\Users\ccana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


    Accuracy Precision    Recall        F1       AUC
SV  0.609524  0.616667  0.466667  0.496667  0.694444
RF  0.638095  0.733333       0.4  0.484762  0.752778
NB  0.466667       0.3       0.1  0.146667  0.469444
    Accuracy Precision    Recall        F1       AUC
SV  0.362573  0.044643  0.714286  0.084034   0.50784
RF  0.467836  0.053191  0.714286   0.09901  0.665505
NB  0.748538  0.071429  0.428571  0.122449  0.663763
    Accuracy Precision    Recall        F1       AUC
SV  0.656126  0.597528  0.955158  0.735117  0.761406
RF  0.945948  0.915756  0.984211  0.948039  0.994851
NB  0.754897  0.699098  0.897053  0.785401  0.815864
    Accuracy Precision    Recall        F1       AUC
SV  0.432749  0.067308       1.0  0.126126  0.830139
RF  0.894737  0.238095  0.714286  0.357143  0.844512
NB   0.54386  0.050633  0.571429  0.093023  0.632404
    Accuracy Precision    Recall        F1       AUC
SV  0.677193  0.687013  0.126316  0.203222  0.745118
RF  0.989474  0.969744       1.0  0.984549  0.

In [ ]:
# Test 1 - P8: NearMiss (se comparan los valores de las matrices cv_results_balanced_nm y test_results_balanced_nm)

In [ ]:
# Test 2 - P8: ADASYN (se comparan los valores de las matrices cv_results_balanced_ad y test_results_balanced_ad)

In [ ]:
# Test 3 - P8: Combinación de técnicas oversampling y undersampling (se comparan los valores de las matrices cv_results_balanced_comb y test_results_balanced_comb)

9. Considerando los resultados obtenidos desde el punto 6 hasta el punto 8 (técnicas de balance de datos), ¿cuál es la mejor técnica de balanceo de datos para este problema? Para responder esta pregunta considere solamente la métrica de desempeño `F1-Score` en el dataset de entrenamiento. 

El nombre del modelo dede ser `best_model_balancing` con una de las siguientes opciones: 'SV', 'RF', 'NB'. Por ejemplo:`best_model_balancing='SV'`.

Además, debe entregar un objeto de tipo `int` con el nombre `best_balancing` que contenga la codificación correspondiente a la mejor técnica de balanceo de datos según la siguiente tabla:

|Técnica|Código|
|-------|------|
|RandomOverSampler|1|
|RandomUnderSampler|2|
|NearMiss|3|
|ADASYN|4|
|Combinación de RandomOverSampler y RandomUnderSampler|5|

Por ejemplo: `best_balancing=1`

In [ ]:
best_model_balancing = None    # Objeto que debe modificar
best_balancing = None          # Objeto que debe modificar

# your code here
raise NotImplementedError

In [ ]:
best_model_balancing = 'RF'
best_balancing = 4


In [ ]:
# Test 1 - P9

10. Considerando los resultados obtenidos desde el punto 6 hasta el punto 8 (técnicas de balance de datos), ¿cuál es la mejor técnica de balanceo de datos para este problema? Para responder esta pregunta considere solamente la métrica de desempeño `F1-Score` en el dataset de test. (Esta comparación es solo confines de comprender la diferencia entre los errores cometidos en la muestra train y test, no se debe seleccionar el modelo desde la muestra test) 

El nombre del modelo dede ser `best_model_balancing_tets` con una de las siguientes opciones: 'SV', 'RF', 'NB'. Por ejemplo:`best_model_balancing_test='SV'`.

Además, debe entregar un objeto de tipo `int` con el nombre `best_balancing_test` que contenga la codificación correspondiente a la mejor técnica de balanceo de datos según la siguiente tabla:

|Técnica|Código|
|-------|------|
|RandomOverSampler|1|
|RandomUnderSampler|2|
|NearMiss|3|
|ADASYN|4|
|Combinación de RandomOverSampler y RandomUnderSampler|5|

Por ejemplo: `best_balancing_test=1`

In [ ]:
best_model_balancing_test = None    # Objeto que debe modificar
best_balancing_test = None          # Objeto que debe modificar

# your code here
raise NotImplementedError

In [53]:
best_model_balancing_test = 'RF'
best_balancing_test = 4  # ADASYN

In [ ]:
# Test 1 - P10

11. Por último, responda las siguientes preguntas, en ambas considere como métrica el `F1-Score`:

a. El mejor método (modelo de clasificación y técnica de balanceo) con la muestra de entrenamiento coincide con el mejor de la muestra test. Para esto, el objeto `resp_a` debe contener algunas de las siguientes:
- 'Los métodos coinciden'
- 'Los métodos coinciden solo en el modelo pero no en la técnica de balanceo'
- 'Los métodos coinciden solo en la técnica de balanceo pero no en el modelo'
- 'Los métodos no coinciden'

Por ejemplo: `resp_a='Los métodos coinciden'`

b. ¿Cuál de las dos métodologias, utilizando los datos originales o utilizando los datos balanceados, tiene mejor generalización en la muestra test. Para esto, el objeto `resp_b` debe contener algunas de las siguientes:
- 'Los métodos tienen prácticamente la misma generalización, es decir, la diferencia es menor a un 1%'
- 'Utilizando los datos originales tiene mejores resultados'
- 'Utilizando los datos balanceados tiene mejores resultados'

Por ejemplo: `resp_b='Los métodos tienen prácticamente la misma generalización, es decir, la diferencia es menor a un 1%'`

In [ ]:
resp_a = None                         # Objeto que debe modificar
resp_b = None                         # Objeto que debe modificar

# your code here
raise NotImplementedError

In [ ]:
resp_a = 'Los métodos coinciden'
resp_b = 'Utilizando los datos balanceados tiene mejores resultados'


In [ ]:
# Test 1 - P11